# Python Packages - for Training Code
### IN ACTIVE DEVELOPMENT - not complete

At the simplest, all the training code may be in a single `filename.py` file that is a module. There are a couple of layers of depth that are commonly added to this:

**Python Modules**

Modules are files: `filename.py`

**Python Project**

Projects are collections of **Python Modules** in folders and possibly subfolders.  Here is an example project named `trainer`.
```bash
│   │   ├── trainer/
│   │   │   ├── __init__.py
│   │   │   ├── train.py
│   │   │   ├── module_1.py
│   │   │   ├── helpers/
│   │   │   │   ├── __init__.py
│   │   │   │   ├── module_a.py
│   │   │   │   ├── module_a.py
```
Here the `train.py` might have `import module_1` and `import helpers.module_a as module_a`.  Note the `__init__.py` file in the folders - this is an empty file that lets Python know the folder can be imported as a module.

**Python Packages**

Packages are creating by adding necessary files to a **Python Project** to help create a distribution package.
```bash
├── training_package/
│   ├── pyproject.toml
│   ├── src/
│   │   ├── trainer/
│   │   │   ├── __init__.py
│   │   │   ├── train.py
│   │   │   ├── module_1.py
│   │   │   ├── helpers/
│   │   │   │   ├── __init__.py
│   │   │   │   ├── module_a.py
│   │   │   │   ├── module_a.py
```

Example `pyproject.toml` file that sets `setuptools` as the build system:
```python
[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = 'trainer'
version = '0.1'
dependencies = ['tensorflow_io', 'google-cloud-aiplatform>=1.17.0']
description = 'Training Package'
authors = [{{name = 'statmike'}}]
```

**Python Distribution Archive**

Prepare **Python Packages** for distribution - called an archive, distribution, or distribution archive. There are two formats for these:
- `file.tar.gz` is a source distributions
    - created with `python setup.py sdist` or `python -m build` run in the package level folder
    - tarballs, `file.tar`, a collection of files wrapped into a single file
    - compressed, `file.tar.gz`, using [gzip](https://www.gzip.org/)
    - contains metadata and source files to be installed by pip
- `.whl` is a built distribution
    - created with `python setup.py bdist_wheel` or `python -m build` from the `package` level folder
    - wheels, `file.whl`, built into a compressed binary format that is portable

Notes on distribution tools:
- here we use the setuptools as the backend build tool specified in the `[build-system]` section of `pyproject.toml`
    - `python -m build` uses `pyproject.toml` to automatically create both `.whl` built distribution and `.tar.gz` source distribution versions
- another way you may see this done is using setuptools directly by creating a `setup.py` file instead of `pyproject.toml`.  It can then be used with setuptools:
    - `python setup.py sdist` which automaticlaly creates `file.tar.gz` by default
    - `python setup.py bdist_wheel` which creates `file.whl`
    - this is the method mentioned on this Vertex AI documentation page for [creating a python training application for a pre-built container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
        - the method in this notebook also builds the source distribution in a compatible way for use with Vertex AI pre-built containers.  You can actually directly use `gzip` to create the source distribution for a folder of training files!
- several advantages to using `build`
    - automatically create source and built distribution in a `/dist` subfolder
    - automatic discovery of modules for common directory structures - [link](https://setuptools.pypa.io/en/latest/userguide/package_discovery.html#automatic-discovery)
    - defaults to include package data files - [link](https://setuptools.pypa.io/en/latest/userguide/datafiles.html#include-package-data)
    - can link to readme.md and license files

**Installing Packages**

When you `pip install ...` what is happening?  This causes pip to look for the package and install it.  The default location to look is [PyPI](https://pypi.org/).  This can be overridden:
- local install `pip install path/to/file.tar.gz` or `pip install path/to/file.whl`
- install from custom repository on Artifact Registry with `pip install --index-url https://{REGION}-python.pkg.dev/{PROJECT}/{REPOSITORY}/{PACKAGE}/ sampleproject`


Resources:
- [pip install](https://pip.pypa.io/en/stable/cli/pip_install/)
- [Packaging Python Projects Tutorial](https://packaging.python.org/en/latest/tutorials/packaging-projects/)
- [setuptools](https://docs.python.org/3/distutils/sourcedist.html)
- [setuptools quickstart](https://setuptools.pypa.io/en/latest/userguide/quickstart.html)

---
## Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry)
- Artifact Registry
    - to create a repository for storing custom Python packages in a GCP Project

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [123]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [124]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    !pip install google-cloud-artifact-registry -q

In [132]:
try:
    import build
except ImportError:
    print('You need to pip install build')
    !pip install build -q

---
## Setup

inputs:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [3]:
REGION = 'us-central1'
EXPERIMENT = 'packages'
SERIES = 'tips'

packages:

In [77]:
import os, shutil
from datetime import datetime
from google.cloud import storage
from google.cloud import aiplatform

clients:

In [22]:
gcs = storage.Client()
aiplatform.init(project = PROJECT_ID, location = REGION)

parameters:

In [6]:
DIR = f'temp/{EXPERIMENT}'

environment:

In [103]:
# remove directory named DIR if exists
shutil.rmtree(DIR, ignore_errors = True)

# create directory DIR
os.makedirs(DIR)

# check for existance of DIR
print('DIR exists? ', os.path.exists(DIR))

# list contents of directory one level higher than DIR
os.listdir(DIR + '/../')

DIR exists?  True


['job-parms', 'gcs', 'multiprocess', 'packages']

## Construct Python Package

Use the temp dirctory crated at DIR:

In [104]:
DIR

'temp/packages'

In [105]:
os.listdir(f'{DIR}/../')

['job-parms', 'gcs', 'multiprocess', 'packages']

### Create the folder structure:

In [106]:
os.makedirs(DIR+'/trainer/src/trainer')

In [107]:
for root, dirs, files in os.walk(DIR):
    print(root)

temp/packages
temp/packages/trainer
temp/packages/trainer/src
temp/packages/trainer/src/trainer


### Add files to directory:

The `05 - TensorFlow` series has a model training file named `05_train.py` that will be used here.

In [108]:
shutil.copyfile('../05 - TensorFlow/05_train.py', f'{DIR}/trainer/src/trainer/train.py')
with open(f'{DIR}/trainer/src/trainer/__init__.py', 'w') as file: pass

In [109]:
with open(f'{DIR}/trainer/pyproject.toml', 'w') as file:
    file.write(f"""[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = 'trainer'
version = '0.1'
# dependencies = ['tensorflow_io', 'google-cloud-aiplatform>={aiplatform.__version__}']
dependencies = ['tensorflow_io', 'google-cloud-aiplatform==1.16.0']
description = 'Training Package'
authors = [{{name = 'statmike'}}]
""")

list directory:

In [110]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/packages/trainer/pyproject.toml
temp/packages/trainer/src/trainer/__init__.py
temp/packages/trainer/src/trainer/train.py


### Build the Python distribution archives:

The build process creates both a `.tar.gz` source distribution and a `.whl` built distribution

In [111]:
!cd ./{DIR}/trainer && python -m build

* Creating virtualenv isolated environment...
* Installing packages in isolated environment... (setuptools)
* Getting dependencies for sdist...
running egg_info
creating src/trainer.egg-info
writing src/trainer.egg-info/PKG-INFO
writing dependency_links to src/trainer.egg-info/dependency_links.txt
writing requirements to src/trainer.egg-info/requires.txt
writing top-level names to src/trainer.egg-info/top_level.txt
writing manifest file 'src/trainer.egg-info/SOURCES.txt'
reading manifest file 'src/trainer.egg-info/SOURCES.txt'
writing manifest file 'src/trainer.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src/trainer.egg-info/PKG-INFO
writing dependency_links to src/trainer.egg-info/dependency_links.txt
writing requirements to src/trainer.egg-info/requires.txt
writing top-level names to src/trainer.egg-info/top_level.txt
reading manifest file 'src/trainer.egg-info/SOURCES.txt'
writing manifest file 'src/trainer.egg-info/SOURCES.txt'

running check
cr

list directory:

In [114]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/packages/trainer/pyproject.toml
temp/packages/trainer/src/trainer.egg-info/top_level.txt
temp/packages/trainer/src/trainer.egg-info/SOURCES.txt
temp/packages/trainer/src/trainer.egg-info/requires.txt
temp/packages/trainer/src/trainer.egg-info/dependency_links.txt
temp/packages/trainer/src/trainer.egg-info/PKG-INFO
temp/packages/trainer/src/trainer/__init__.py
temp/packages/trainer/src/trainer/train.py
temp/packages/trainer/dist/trainer-0.1-py3-none-any.whl
temp/packages/trainer/dist/trainer-0.1.tar.gz


**Review**

This single folder now has:

- a single training file/module: {DIR}/training/src/trainer/train.py
- a folder of training code: {DIR}/training/src/trainer*
    - with a starting point of train.py
- a source distribution: {DIR}/training/dist/trainer-0.1.tar.gz
- a built distribution: {DIR}/training/dist/trainer-0.1-py3-none-any.whl

### Copy to GCS

Here the folder structure for DIR will be copied to the GCS Bucket used across this project.  This section uses skills that are discussed in more detail in the [Python Client for GCS](./Python%20Client%20for%20GCS.ipynb) notebook.

List buckets in project:

In [115]:
list(gcs.list_buckets())

[<Bucket: cloud-ai-platform-a68e7f3a-fac8-47f6-9f92-fff95c09cdb8>,
 <Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>]

Get the bucket:

In [116]:
bucket = gcs.lookup_bucket(PROJECT_ID)

list file to upload:

In [117]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/packages/trainer/pyproject.toml
temp/packages/trainer/src/trainer.egg-info/top_level.txt
temp/packages/trainer/src/trainer.egg-info/SOURCES.txt
temp/packages/trainer/src/trainer.egg-info/requires.txt
temp/packages/trainer/src/trainer.egg-info/dependency_links.txt
temp/packages/trainer/src/trainer.egg-info/PKG-INFO
temp/packages/trainer/src/trainer/__init__.py
temp/packages/trainer/src/trainer/train.py
temp/packages/trainer/dist/trainer-0.1-py3-none-any.whl
temp/packages/trainer/dist/trainer-0.1.tar.gz


list of desired bucket object URIs:

In [118]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        filepath = os.path.join(root, f)
        gcspath = f'{SERIES}/{EXPERIMENT}{filepath[len(DIR):]}'
        print(gcspath)

tips/packages/trainer/pyproject.toml
tips/packages/trainer/src/trainer.egg-info/top_level.txt
tips/packages/trainer/src/trainer.egg-info/SOURCES.txt
tips/packages/trainer/src/trainer.egg-info/requires.txt
tips/packages/trainer/src/trainer.egg-info/dependency_links.txt
tips/packages/trainer/src/trainer.egg-info/PKG-INFO
tips/packages/trainer/src/trainer/__init__.py
tips/packages/trainer/src/trainer/train.py
tips/packages/trainer/dist/trainer-0.1-py3-none-any.whl
tips/packages/trainer/dist/trainer-0.1.tar.gz


upload files as objects:

In [119]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        filepath = os.path.join(root, f)
        gcspath = f'{SERIES}/{EXPERIMENT}{filepath[len(DIR):]}'
        blob = bucket.blob(gcspath)
        blob.upload_from_filename(filepath)

In [120]:
print(f"View the bucket directly here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}")

View the bucket directly here:
https://console.cloud.google.com/storage/browser/statmike-mlops-349915;tab=objects&project=statmike-mlops-349915


list files in bucket:

In [122]:
for blob in list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/trainer')):
    print(blob.name)

tips/packages/trainer/dist/trainer-0.1-py3-none-any.whl
tips/packages/trainer/dist/trainer-0.1.tar.gz
tips/packages/trainer/pyproject.toml
tips/packages/trainer/src/trainer.egg-info/PKG-INFO
tips/packages/trainer/src/trainer.egg-info/SOURCES.txt
tips/packages/trainer/src/trainer.egg-info/dependency_links.txt
tips/packages/trainer/src/trainer.egg-info/requires.txt
tips/packages/trainer/src/trainer.egg-info/top_level.txt
tips/packages/trainer/src/trainer/__init__.py
tips/packages/trainer/src/trainer/train.py


## Artifact Registry

- describe
- setup
- create repository for python
- upload package
- show listing at multiple levels - see doc

## Vertex AI Training - Custom Jobs

Vertex AI Training has Custom Jobs that can be launched with:
- single files/modules from local disk
- source distributions from GCS URI's
- custom containers

Below are tests/examples for the single file and source distribution created in this notebook.  The custom container workflows will be examined further in the [Python Client for Cloud Build](./Python%20Client%20for%20Cloud%20Build.ipynb) notebook.

### Job Inputs

The example test jobs below are based on jobs in the `05 - TensorFlow` series and takes advantage of Vertex AI Experiments and mangaed TensorBoard.  This section creates a TensorBoard instance and gets other inputs for the jobs:

In [86]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

Creating Tensorboard
Create Tensorboard backing LRO: projects/1026793852137/locations/us-central1/tensorboards/7360834523774320640/operations/7276711232730562560
Tensorboard created. Resource name: projects/1026793852137/locations/us-central1/tensorboards/7360834523774320640
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/1026793852137/locations/us-central1/tensorboards/7360834523774320640')


In [87]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/7360834523774320640'

In [88]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

### Custom Job: From Local Script

This is a modified version of notebook [05a - Vertex AI Custom Model - TensorFlow - Custom Job With Python File](../05%20-%20TensorFlow/05a%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20File.ipynb) that uses the local script for this project.

Notes:
- This uses a single `file.py` from the local directory, not a GCS URI
- When you run `aiplatform.CustomJob.from_local_script()` it responds with a message confirming the local script was copied to the GCS URI provide in the parameter `staging_bucket = `.

In [95]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-tf-classification-dnn'
RUN_NAME = f'run-{TIMESTAMP}'

TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
URI = f'gs://{PROJECT_ID}/{SERIES}/{EXPERIMENT}/models'


CMDARGS = [
    "--epochs=5",
    "--batch_size=100",
    "--var_target=Class",
    "--var_omit=transaction_id",
    f"--project_id={PROJECT_ID}",
    f"--bq_project={PROJECT_ID}",
    "--bq_dataset=fraud",
    "--bq_table=fraud_prepped",
    f"--region={REGION}",
    f"--experiment={EXPERIMENT}",
    f"--series={SERIES}",
    f"--experiment_name={EXPERIMENT_NAME}",
    f"--run_name={RUN_NAME}"
]

In [96]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

In [98]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    script_path = f"{DIR}/trainer/src/trainer/train.py",
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['tensorflow_io', 'google-cloud-aiplatform==1.16.0'],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

Training script copied to:
gs://statmike-mlops-349915/tips/packages/models/20220920133849/aiplatform-2022-09-20-13:38:54.543-aiplatform_custom_trainer_script-0.1.tar.gz.


In [99]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/7289745324602556416
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/7289745324602556416')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7289745324602556416?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7360834523774320640+experiments+7289745324602556416
CustomJob projects/1026793852137/locations/us-central1/customJobs/7289745324602556416 current state:
JobState.JOB_STATE_QUEUED
CustomJob projects/1026793852137/locations/us-central1/customJobs/7289745324602556416 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/7289745324602556416 current state:
JobState.JOB_STATE_PENDING
C

In [100]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
print(f'Review the Job here:\n{job_link}')

Review the Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/7289745324602556416/cpu?cloudshell=false&project=statmike-mlops-349915


In [131]:
print(f'Review the model output here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}?project={PROJECT_ID}')

Review the model output here:
https://console.cloud.google.com/storage/browser/statmike-mlops-349915/tips/packages/models/20220920141208?project=statmike-mlops-349915


### Custom Job: From Python Source Distribution

This is a modified version of notebook [05b - Vertex AI Custom Model - TensorFlow - Custom Job With Python Source Distribution](../05%20-%20TensorFlow/05b%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20Source%20Distribution.ipynb) that uses the source distribution stored in GCS for this project.

Notes:

In [123]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-tf-classification-dnn'
RUN_NAME = f'run-{TIMESTAMP}'

TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
URI = f'gs://{PROJECT_ID}/{SERIES}/{EXPERIMENT}/models'

CMDARGS = [
    "--epochs=5",
    "--batch_size=100",
    "--var_target=Class",
    "--var_omit=transaction_id",
    f"--project_id={PROJECT_ID}",
    f"--bq_project={PROJECT_ID}",
    "--bq_dataset=fraud",
    "--bq_table=fraud_prepped",
    f"--region={REGION}",
    f"--experiment={EXPERIMENT}",
    f"--series={SERIES}",
    f"--experiment_name={EXPERIMENT_NAME}",
    f"--run_name={RUN_NAME}"
]

MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [f"gs://{PROJECT_ID}/{SERIES}/{EXPERIMENT}/trainer/dist/trainer-0.1.tar.gz"],
            "python_module": "trainer.train",
            "args": CMDARGS
        }
    }
]

In [124]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

In [125]:
customJob = aiplatform.CustomJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

In [126]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/2326778535240269824
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/2326778535240269824')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2326778535240269824?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7360834523774320640+experiments+2326778535240269824
CustomJob projects/1026793852137/locations/us-central1/customJobs/2326778535240269824 current state:
JobState.JOB_STATE_QUEUED
CustomJob projects/1026793852137/locations/us-central1/customJobs/2326778535240269824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/2326778535240269824 current state:
JobState.JOB_STATE_PENDING
C

In [128]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
print(f'Review the Job here:\n{job_link}')

Review the Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/2326778535240269824/cpu?cloudshell=false&project=statmike-mlops-349915


In [130]:
print(f'Review the model output here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}?project={PROJECT_ID}')

Review the model output here:
https://console.cloud.google.com/storage/browser/statmike-mlops-349915/tips/packages/models/20220920141208?project=statmike-mlops-349915
